In [2]:
print("hi")
from transformers import TFBertModel, BertTokenizer,BertConfig,BertModel
import re
import numpy as np
import torch

hi


/home/pavit21178/anaconda3/envs/my_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:


# Load the ProtBert tokenizer
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert_bfd", do_lower_case=False)

# Load the ProtBert model
model = BertModel.from_pretrained("Rostlab/prot_bert_bfd")

# Set model to evaluation mode
model.eval()
import pandas as pd
df=pd.read_csv('merged_data_modified.csv')
y=df['id']
x=df['seq']

#split the data into train and test





tokenizer_config.json: 100%|██████████| 86.0/86.0 [00:00<00:00, 8.73kB/s]
vocab.txt: 100%|██████████| 81.0/81.0 [00:00<00:00, 6.50kB/s]
config.json: 100%|██████████| 361/361 [00:00<00:00, 124kB/s]
pytorch_model.bin: 100%|██████████| 1.68G/1.68G [02:27<00:00, 11.4MB/s]


In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#oversample the data to create a balanced dataset in train
#use random oversampling
print(x_train.shape)
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
x_train=x_train.values.reshape(-1,1)
x_train, y_train = ros.fit_resample(x_train, y_train)
x_train=x_train.reshape(-1)




(353,)


In [10]:
print(x_train.shape,y_train.shape)

(494,) (494,)


In [11]:


ids = tokenizer.batch_encode_plus(x_train, max_length=40, add_special_tokens=True, pad_to_max_length=True, return_tensors="pt")
input_ids = ids['input_ids']
attention_mask = ids['attention_mask']  

# Get the hidden states of the BERT model
outputs= model(input_ids, attention_mask=attention_mask)
hidden_states = outputs.last_hidden_state


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/pavit21178/anaconda3/envs/my_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [12]:
print(hidden_states.shape)

torch.Size([494, 40, 1024])


In [13]:
#do same with test data
ids = tokenizer.batch_encode_plus(x_test, max_length=40, add_special_tokens=True, pad_to_max_length=True, return_tensors="pt")
test_input_ids = ids['input_ids']
test_attention_mask = ids['attention_mask']

# Get the hidden states of the BERT model
test_outputs= model(test_input_ids, attention_mask=test_attention_mask)
test_hidden_states = test_outputs.last_hidden_state

print(test_hidden_states.shape)

/home/pavit21178/anaconda3/envs/my_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


torch.Size([89, 40, 1024])


In [14]:
print(test_hidden_states.shape)


torch.Size([89, 40, 1024])


In [15]:
import tensorflow as tf
def build_CNN_classifier_model():
    input = tf.keras.Input(shape=(40,1024)) # [batch_size, length, 1024].
    #net = input # [batch_size, seq_length, 1024]


    net = tf.keras.layers.Conv1D(128, (2), activation='relu', data_format='channels_last')(input)
    net = tf.keras.layers.Conv1D(64, (1), activation='relu', data_format='channels_last')(net)

    net = tf.keras.layers.GlobalMaxPool1D()(net)

#    net = tf.keras.layers.Flatten()(net)

    net = tf.keras.layers.Dense(32, activation="relu")(net)

    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(2, activation='softmax')(net)

    return tf.keras.Model(input, net)

2024-04-03 01:28:20.935795: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
#run the model
model = build_CNN_classifier_model()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(hidden_states, y_train, epochs=10, batch_size=32, validation_data=(test_hidden_states, y_test))



Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 40, 1024)]        0         
                                                                 
 conv1d_4 (Conv1D)           (None, 39, 128)           262272    
                                                                 
 conv1d_5 (Conv1D)           (None, 39, 64)            8256      
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                           

TypeError: Cannot convert the argument `type_value`: torch.float32 to a TensorFlow DType.

In [17]:
bitter_classifier = build_CNN_classifier_model()
output_test = bitter_classifier(test_outputs) # tensor [batch_size, max_length, 1024]
output_test

ValueError: Missing data for input "input_2". You passed a data dictionary with keys ['last_hidden_state', 'pooler_output']. Expected the following keys: ['input_2']